In [1]:
import quandl
import csv
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
key = '2ubHBg4pkVHEn5ivHGsL'
quandl.ApiConfig.api_key = key

In [196]:
biotech = pd.read_csv("85_biotech_companies_timeseries.csv",index_col=0)
biotech = biotech.sort_values('Date').reset_index()
biotech

,index,Date,YearMonth,ACAD_Open,ACAD_High,ACAD_Low,ACAD_Close,ACAD_Volume,ACAD_UpOrDownMonthEnd,XLRN_Open,...,XNCR_Low,XNCR_Close,XNCR_Volume,XNCR_UpOrDownMonthEnd,PFE_Open,PFE_High,PFE_Low,PFE_Close,PFE_Volume,PFE_UpOrDownMonthEnd
0,8458,1972-06-01,Jun-1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,41.22,41.2500,40.62,41.25,48600.0,1.0
1,8459,1972-06-02,Jun-1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,41.25,41.3800,40.62,40.75,31900.0,1.0
2,8460,1972-06-05,Jun-1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,40.75,40.8700,40.38,40.62,51100.0,1.0
3,8461,1972-06-06,Jun-1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,40.62,41.7500,40.50,41.50,46400.0,1.0
4,8462,1972-06-07,Jun-1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,41.50,41.5000,40.87,41.50,20400.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11552,3476,2018-03-21,Mar-2018,24.53,24.9400,24.250,24.66,1105427.0,0.0,42.21,...,29.53,30.98,3901210.0,0.0,36.49,36.7300,36.20,36.27,16405382.0,0.0
11553,3477,2018-03-22,Mar-2018,24.45,24.7890,23.760,23.84,1243951.0,0.0,42.85,...,30.00,30.32,564748.0,0.0,36.03,36.1433,35.47,35.60,20288017.0,0.0
11554,3478,2018-03-23,Mar-2018,23.63,23.9500,22.470,22.57,2387381.0,0.0,41.36,...,29.95,30.44,321017.0,0.0,35.49,35.5000,34.44,34.49,27489780.0,0.0
11555,3479,2018-03-26,Mar-2018,23.02,23.6400,22.540,23.54,1612893.0,0.0,40.29,...,30.24,31.08,801972.0,0.0,34.93,35.1500,34.32,35.04,23464967.0,0.0


In [197]:
#define function to interpolate across a column
def interpolate(array):
    #scan once up to the first value, interpolate, and put back into array,input is list
    first_non_nan_idx = None
    for i in range(0,len(array)):
        if np.isnan(array[i]) == False:
            last_nan_idx = i
            break
    for i in range(0,last_nan_idx):
        array[i] = (i / last_nan_idx)*array[last_nan_idx]
    return array

#apply over the whole function
cols = [col for col in biotech.columns[3:] if 'End' not in col]
for c in cols:
    array = interpolate(list(biotech[c]))
    biotech[c] = array
#fill in missling labels, for the previous value we imputed, since they aren't technically down, classify as up
#so fill in ones
cols = [col for col in biotech.columns[3:] if 'End' in col]
for c in cols:
    biotech[c] = biotech[c].fillna(value=1.0)
    'PFE_UpOrDownMonthEnd'
cols = [col for col in biotech.columns[1:] if 'End' not in col]
cols += ['PFE_UpOrDownMonthEnd']
biotech = biotech[cols]
biotech = biotech.dropna()
biotech

,Date,YearMonth,ACAD_Open,ACAD_High,ACAD_Low,ACAD_Close,ACAD_Volume,XLRN_Open,XLRN_High,XLRN_Low,...,XNCR_High,XNCR_Low,XNCR_Close,XNCR_Volume,PFE_Open,PFE_High,PFE_Low,PFE_Close,PFE_Volume,PFE_UpOrDownMonthEnd
0,1972-06-01,Jun-1972,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,41.22,41.25,40.62,41.25,48600.0,1.0
1,1972-06-02,Jun-1972,0.000921,0.000929,0.000805,0.000830,2.376424e+02,0.001775,0.001996,0.001775,...,0.000809,0.000549,0.000796,179.478610,41.25,41.38,40.62,40.75,31900.0,1.0
2,1972-06-05,Jun-1972,0.001842,0.001857,0.001610,0.001659,4.752848e+02,0.003551,0.003992,0.003551,...,0.001618,0.001098,0.001593,358.957219,40.75,40.87,40.38,40.62,51100.0,1.0
3,1972-06-06,Jun-1972,0.002764,0.002786,0.002415,0.002489,7.129272e+02,0.005326,0.005988,0.005326,...,0.002426,0.001647,0.002389,538.435829,40.62,41.75,40.50,41.50,46400.0,1.0
4,1972-06-07,Jun-1972,0.003685,0.003715,0.003219,0.003318,9.505696e+02,0.007102,0.007985,0.007102,...,0.003235,0.002196,0.003186,717.914439,41.50,41.50,40.87,41.50,20400.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10795,2015-03-18,Mar-2015,33.810000,34.640000,33.470000,34.460000,1.869529e+06,41.210000,41.818000,40.500000,...,15.630000,15.160000,15.350000,183631.000000,34.03,34.50,33.75,34.38,33132522.0,1.0
10796,2015-03-19,Mar-2015,34.430000,35.446700,34.080000,34.990000,2.196661e+06,41.450000,42.775000,41.310100,...,16.000000,15.430000,15.990000,201362.000000,34.28,34.38,34.07,34.15,20437408.0,1.0
11554,2018-03-23,Mar-2018,23.630000,23.950000,22.470000,22.570000,2.387381e+06,41.360000,41.810000,39.375000,...,31.180000,29.950000,30.440000,321017.000000,35.49,35.50,34.44,34.49,27489780.0,0.0
11555,2018-03-26,Mar-2018,23.020000,23.640000,22.540000,23.540000,1.612893e+06,40.290000,41.520000,39.250000,...,31.330000,30.240000,31.080000,801972.000000,34.93,35.15,34.32,35.04,23464967.0,0.0


In [183]:
pfizer = biotech.iloc[:,-6:428]
pfizer['YearMonth'] = biotech['YearMonth']
pfizer = pfizer.dropna()
pfizer

,PFE_Open,PFE_High,PFE_Low,PFE_Close,PFE_Volume,PFE_UpOrDownMonthEnd,YearMonth
0,41.22,41.2500,40.62,41.25,48600.0,1.0,Jun-1972
1,41.25,41.3800,40.62,40.75,31900.0,1.0,Jun-1972
2,40.75,40.8700,40.38,40.62,51100.0,1.0,Jun-1972
3,40.62,41.7500,40.50,41.50,46400.0,1.0,Jun-1972
4,41.50,41.5000,40.87,41.50,20400.0,1.0,Jun-1972
...,...,...,...,...,...,...,...
11552,36.49,36.7300,36.20,36.27,16405382.0,0.0,Mar-2018
11553,36.03,36.1433,35.47,35.60,20288017.0,0.0,Mar-2018
11554,35.49,35.5000,34.44,34.49,27489780.0,0.0,Mar-2018
11555,34.93,35.1500,34.32,35.04,23464967.0,0.0,Mar-2018


In [201]:
shapes = []
for a,b in biotech.groupby('YearMonth'):
    shapes.append(b.shape)
shapes

[(20, 428),
 (21, 428),
 (22, 428),
 (21, 428),
 (20, 428),
 (20, 428),
 (20, 428),
 (21, 428),
 (21, 428),
 (21, 428),
 (20, 428),
 (20, 428),
 (21, 428),
 (22, 428),
 (21, 428),
 (20, 428),
 (20, 428),
 (20, 428),
 (22, 428),
 (21, 428),
 (21, 428),
 (19, 428),
 (19, 428),
 (21, 428),
 (22, 428),
 (21, 428),
 (21, 428),
 (19, 428),
 (20, 428),
 (22, 428),
 (21, 428),
 (21, 428),
 (21, 428),
 (19, 428),
 (20, 428),
 (22, 428),
 (21, 428),
 (21, 428),
 (20, 428),
 (20, 428),
 (22, 428),
 (21, 428),
 (23, 428),
 (23, 428),
 (22, 428),
 (21, 428),
 (22, 428),
 (23, 428),
 (23, 428),
 (23, 428),
 (21, 428),
 (21, 428),
 (22, 428),
 (23, 428),
 (23, 428),
 (22, 428),
 (21, 428),
 (21, 428),
 (23, 428),
 (23, 428),
 (23, 428),
 (22, 428),
 (21, 428),
 (22, 428),
 (23, 428),
 (23, 428),
 (22, 428),
 (21, 428),
 (21, 428),
 (22, 428),
 (23, 428),
 (23, 428),
 (22, 428),
 (21, 428),
 (22, 428),
 (23, 428),
 (23, 428),
 (23, 428),
 (21, 428),
 (21, 428),
 (22, 428),
 (23, 428),
 (23, 428),
 (22

In [204]:
X_s = np.zeros((550,23,425))
y_s = np.zeros((550,23))
idx = 0
for a,b in biotech.groupby('YearMonth'):
    #pull corresposnidng X and y
    X,y = b.iloc[:,2:-1].values, b.iloc[:,-1].values
    #is less than 23
    if X.shape[0] < 23:
        #concat with the last number of rows
        for i in range(23-X.shape[0]):
            X = np.vstack((X,X[-1,:]))
            y = np.hstack((y,y[-1].reshape((1,))))
    X_s[idx,:,:] = X
    y_s[idx,:] = y
    idx += 1

In [205]:
X_s[5,:,:]

array([[1.35515602e+00, 1.36608470e+00, 1.18394007e+00, ...,
        2.73700000e+01, 2.73700000e+01, 4.11000000e+04],
       [1.35607727e+00, 1.36701337e+00, 1.18474492e+00, ...,
        2.73700000e+01, 2.78800000e+01, 6.04000000e+04],
       [1.35699851e+00, 1.36794205e+00, 1.18554978e+00, ...,
        2.73700000e+01, 2.78800000e+01, 7.06000000e+04],
       ...,
       [1.37265973e+00, 1.38372957e+00, 1.19923229e+00, ...,
        2.92500000e+01, 3.02500000e+01, 1.26600000e+05],
       [1.37265973e+00, 1.38372957e+00, 1.19923229e+00, ...,
        2.92500000e+01, 3.02500000e+01, 1.26600000e+05],
       [1.37265973e+00, 1.38372957e+00, 1.19923229e+00, ...,
        2.92500000e+01, 3.02500000e+01, 1.26600000e+05]])

In [206]:
X_s.shape,y_s.shape

((550, 23, 425), (550, 23))

In [207]:
#shuffle
from sklearn.utils import shuffle
X_s,y_s = shuffle(X_s,y_s)

In [208]:
X_s[0,:,:]

array([[4.05533432e+00, 4.08803863e+00, 3.54296682e+00, ...,
        6.77500000e+01, 6.80000000e+01, 7.51900000e+05],
       [4.05625557e+00, 4.08896731e+00, 3.54377167e+00, ...,
        6.66200000e+01, 6.77500000e+01, 5.81100000e+05],
       [4.05717682e+00, 4.08989599e+00, 3.54457652e+00, ...,
        6.70000000e+01, 6.80000000e+01, 4.70300000e+05],
       ...,
       [4.07375929e+00, 4.10661218e+00, 3.55906389e+00, ...,
        7.33700000e+01, 7.41200000e+01, 4.74400000e+05],
       [4.07375929e+00, 4.10661218e+00, 3.55906389e+00, ...,
        7.33700000e+01, 7.41200000e+01, 4.74400000e+05],
       [4.07375929e+00, 4.10661218e+00, 3.55906389e+00, ...,
        7.33700000e+01, 7.41200000e+01, 4.74400000e+05]])

In [209]:
from keras.models import Model
from keras.layers import Input,Dense
from keras.layers import LSTM
import tensorflow as tf
import keras
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional
from keras.layers import TimeDistributed, BatchNormalization

# LSTM units
units = 3

# define model
inputs1 = Input(shape=(23, 425))
lstm1, state_h, state_c = LSTM(32, return_sequences=True, return_state=True)(inputs1)
hidden_state_preds = Dense(1,activation='sigmoid')(lstm1)
last_step_preds = Dense(1,activation='sigmoid')(state_h)
model = Model(inputs=inputs1, outputs=[hidden_state_preds,last_step_preds])

output = model.predict(X_s[107,:,:].reshape((1,23,425)))
print(output)

[array([[[0.5041794 ],
        [0.48766488],
        [0.48513517],
        [0.48478672],
        [0.48473942],
        [0.48473302],
        [0.48473215],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473203],
        [0.48473206],
        [0.48473206],
        [0.48473206]]], dtype=float32), array([[0.407329]], dtype=float32)]


In [210]:
# compile the model
model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(lr=0.000001),
    metrics=["accuracy"],
)

# keep track of the results
history = model.fit(X_s,
                    [y_s.reshape((550,23,1)),y_s[:,-1]], 
                    batch_size=32,
                    epochs=5)

Epoch 1/5
550/550 [==============================] - 11s 20ms/step - loss: 1.4436 - dense_38_loss: 0.7097 - dense_39_loss: 0.7339 - dense_38_acc: 0.5108 - dense_39_acc: 0.5055
Epoch 2/5
550/550 [==============================] - 1s 2ms/step - loss: 1.4436 - dense_38_loss: 0.7097 - dense_39_loss: 0.7339 - dense_38_acc: 0.5213 - dense_39_acc: 0.5145
Epoch 3/5
550/550 [==============================] - 1s 2ms/step - loss: 1.4436 - dense_38_loss: 0.7097 - dense_39_loss: 0.7339 - dense_38_acc: 0.5240 - dense_39_acc: 0.5145
Epoch 4/5
550/550 [==============================] - 1s 2ms/step - loss: 1.4435 - dense_38_loss: 0.7097 - dense_39_loss: 0.7338 - dense_38_acc: 0.5240 - dense_39_acc: 0.5145
Epoch 5/5
550/550 [==============================] - 1s 2ms/step - loss: 1.4435 - dense_38_loss: 0.7097 - dense_39_loss: 0.7338 - dense_38_acc: 0.5239 - dense_39_acc: 0.5145


### done in another way

In [ ]:
X_input = Input(shape = (23, 425))

X = LSTM(units = 256, 
         return_sequences = True)(X_input)          
X = Dropout(0.2)(X)                                       
X = BatchNormalization()(X)                               
X = Dropout(0.2)(X)                                       

X = TimeDistributed(Dense(1, activation = "sigmoid"))(X)
model = Model(inputs = X_input, outputs = X)
opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
# keep track of the results
history = model.fit(X_s,
                    y_s.reshape((550,23,1)), 
                    batch_size=32,
                    epochs=1000)


Epoch 1/1000
550/550 [==============================] - 16s 29ms/step - loss: 0.8183 - acc: 0.5311
Epoch 2/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.8171 - acc: 0.5349
Epoch 3/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.8062 - acc: 0.5368
Epoch 4/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.8116 - acc: 0.5393
Epoch 5/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.8143 - acc: 0.5410
Epoch 6/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.8100 - acc: 0.5329
Epoch 7/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.8044 - acc: 0.5381
Epoch 8/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.8048 - acc: 0.5496
Epoch 9/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.8052 - acc: 0.5386
Epoch 10/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.8071 - acc: 0.5319
Epoch 11/1000
550/550 [====

550/550 [==============================] - 3s 5ms/step - loss: 0.7888 - acc: 0.5421
Epoch 85/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7834 - acc: 0.5417
Epoch 86/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7819 - acc: 0.5444
Epoch 87/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7756 - acc: 0.5452
Epoch 88/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7778 - acc: 0.5483
Epoch 89/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7815 - acc: 0.5439
Epoch 90/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7750 - acc: 0.5504
Epoch 91/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7841 - acc: 0.5466
Epoch 92/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7850 - acc: 0.5449
Epoch 93/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7782 - acc: 0.5447
Epoch 94/1000
550/550 [===========

550/550 [==============================] - 2s 4ms/step - loss: 0.7859 - acc: 0.5406
Epoch 168/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7743 - acc: 0.5467
Epoch 169/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7817 - acc: 0.5419
Epoch 170/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7759 - acc: 0.5485
Epoch 171/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7786 - acc: 0.5454
Epoch 172/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7704 - acc: 0.5480
Epoch 173/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7798 - acc: 0.5465
Epoch 174/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7755 - acc: 0.5416
Epoch 175/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7746 - acc: 0.5488
Epoch 176/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7725 - acc: 0.5447
Epoch 177/1000
550/550 [=

550/550 [==============================] - 3s 5ms/step - loss: 0.7652 - acc: 0.5456
Epoch 250/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7709 - acc: 0.5468
Epoch 251/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7741 - acc: 0.5424
Epoch 252/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7785 - acc: 0.5387
Epoch 253/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7718 - acc: 0.5406
Epoch 254/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7633 - acc: 0.5511
Epoch 255/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7732 - acc: 0.5489
Epoch 256/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7767 - acc: 0.5470
Epoch 257/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7739 - acc: 0.5490
Epoch 258/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7670 - acc: 0.5423
Epoch 259/1000
550/550 [=

550/550 [==============================] - 2s 4ms/step - loss: 0.7842 - acc: 0.5362
Epoch 332/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7681 - acc: 0.5432
Epoch 333/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7671 - acc: 0.5470
Epoch 334/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7673 - acc: 0.5467
Epoch 335/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7765 - acc: 0.5445
Epoch 336/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7729 - acc: 0.5404
Epoch 337/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7725 - acc: 0.5425
Epoch 338/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7809 - acc: 0.5336
Epoch 339/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7677 - acc: 0.5472
Epoch 340/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7705 - acc: 0.5505
Epoch 341/1000
550/550 [=

550/550 [==============================] - 2s 5ms/step - loss: 0.7734 - acc: 0.5464
Epoch 414/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7646 - acc: 0.5489
Epoch 415/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7714 - acc: 0.5423
Epoch 416/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7752 - acc: 0.5435
Epoch 417/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7755 - acc: 0.5419
Epoch 418/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7740 - acc: 0.5453
Epoch 419/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7737 - acc: 0.5416
Epoch 420/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7720 - acc: 0.5454
Epoch 421/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7702 - acc: 0.5435
Epoch 422/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7681 - acc: 0.5446
Epoch 423/1000
550/550 [=

550/550 [==============================] - 3s 5ms/step - loss: 0.7769 - acc: 0.5384
Epoch 496/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7667 - acc: 0.5417
Epoch 497/1000
550/550 [==============================] - 2s 5ms/step - loss: 0.7725 - acc: 0.5419
Epoch 498/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7724 - acc: 0.5455
Epoch 499/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7634 - acc: 0.5484
Epoch 500/1000
550/550 [==============================] - 3s 5ms/step - loss: 0.7673 - acc: 0.5487
Epoch 501/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7645 - acc: 0.5491
Epoch 502/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7688 - acc: 0.5465
Epoch 503/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7726 - acc: 0.5451
Epoch 504/1000
550/550 [==============================] - 2s 4ms/step - loss: 0.7671 - acc: 0.5421
Epoch 505/1000
550/550 [=

In [220]:
history.history

{'loss': [0.981940201846036,
  0.9618398410623724,
  0.9042992336099798,
  0.8786923302303661,
  0.8690719749710777,
  0.8439839597181841,
  0.8540864575992931,
  0.8396498851342634,
  0.8257068178870461,
  0.8372698512944308,
  0.8052193268862637,
  0.8105554881962863,
  0.8171541157635775,
  0.7986039235375144,
  0.7998045264590871,
  0.7995237265933643,
  0.7865102397311817,
  0.7927859321507541,
  0.791221367445859,
  0.7811082415147261,
  0.7867212013764815,
  0.7756133255091581,
  0.7745072425495495,
  0.785482104474848,
  0.7925400194254788,
  0.7780334600535306,
  0.764567148251967,
  0.7757455945014954,
  0.7783813565427606,
  0.7559728052399375,
  0.755483417077498,
  0.7725373790480874,
  0.767310893535614,
  0.7642072029547258,
  0.7643988065286116,
  0.7630531666495584,
  0.7539159367301247,
  0.7564839785749262,
  0.7526747070659291,
  0.7759176230430603,
  0.7521291067383507,
  0.755011845068498,
  0.7584016873619773,
  0.7511300097812306,
  0.7510802856358615,
  0.74785

In [90]:
y_s[126,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [92]:
biotech['PFE_UpOrDownMonthEnd'].value_counts()

1.0    6058
0.0    5499
Name: PFE_UpOrDownMonthEnd, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(biotech.iloc[:,2:-1])
biotech.iloc[:,2:-1] = scaler.transform(biotech.iloc[:,2:-1])